# For GrayScale Image

In [2]:
import pandas as pd
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time
import csv



class Image:

    def __init__(self) -> None:
        pass   
            
    def get_block(self,image,rows,columns):
        height, width = image.shape

        # Variable declared for current height,current width,new height andd new width
        curr_hei = 0 
        curr_wid = 0
        new_hei = height//rows
        new_wid = width//columns

        # for storing each block in list
        block_list = []

        # hei_ref is used to move from current row to next row
        hei_ref = new_hei

        # For getting only first block --> 1st row, 1st column block 
        if curr_wid == 0:
            crop_blk = image[curr_hei:new_hei,curr_wid:new_wid]

            # Store only first block 
            block_list.append(crop_blk)

        # wid_ref is used to move from current column to next column
        wid_ref = new_wid

        # Getting block from 1st row and from 2nd column to column passed of first row   --> 1st rows and 2,3 column as rows = 4 and columns = 3 in example
        # Here we get 2 block from 1st row and remain 2,3 columns because we have already store 1st row and 1st column block in block_list
        for j in range(2,columns+1):
            curr_wid = new_wid
            new_wid = wid_ref*j
            crop_blk = image[curr_hei:new_hei,curr_wid:new_wid]

            # Store 2 block for 1st row and remain 2nd,3rd column of first row , 1st row and 2nd,3rd columns
            block_list.append(crop_blk) 

        # Getting block from 2nd row , each columns and repeat upto rows passed --> Here rows = 2,3,4 and columns = 1,2,3
        for i in range(0,rows):
            if i > 0:
                row_inc = i+1
                curr_hei = new_hei
                new_hei = hei_ref * row_inc
                curr_wid = 0
                new_wid = new_wid//columns

                # Repeat same process of previous code for remaining row and column
                if curr_wid == 0:
                    crop_blk = image[curr_hei:new_hei,curr_wid:new_wid]
                    block_list.append(crop_blk)
                n = new_wid
                for j in range(2,columns+1):
                    curr_wid = new_wid
                    new_wid = wid_ref*j
                    crop_blk = image[curr_hei:new_hei,curr_wid:new_wid]
                    block_list.append(crop_blk)
        return block_list 

    def avg_block_diff(self,block1,block2):

        # Get average of first image block here master image
        avg_block1 = np.mean(block1)

        # Get average of second image block here csv2 file image 
        avg_block2 = np.mean(block2)

        # Get average of master block and image block
        diff_avg = avg_block1 - avg_block2
        return diff_avg

    def avg_master_diff(self,block1):

        # Get max value of master block
        mst_max = np.max(block1)

        # Get minimum value of master block
        mst_min = np.min(block1)

        # Difference between maximum and minimum value of master block
        max_min = mst_max - mst_min
        return max_min

    def align(self,refx,refy,x1,y1,img):
        rows, cols = img.shape
        if (refx==x1) and (refy==y1):
            #print('Same')
            return img
            #plt.imshow(img)
        else:
            difx= refx - x1
            dify = refy - y1
            if (dify>0):
                crop_img = img[0:rows,(cols-dify):cols]
                img = img[0:rows,0:(cols-dify)]
                im_h = cv2.hconcat([crop_img, img])
                #plt.imshow(im_h)
                img = im_h
                #return im_h
            elif (dify<0):
                dify = abs(dify)
                crop_img = img[0:rows,0:dify]
                img = img[0:rows,dify:cols]
                im_h = cv2.hconcat([img,crop_img])
                img = im_h
            else:
                img = img
                #plt.imshow(im_h)
                #return im_h
            if (difx>0):
                crop_img = img[0:difx,0:cols]
                img = img[difx:rows,0:cols]
                im_v = cv2.vconcat([img,crop_img])
                img = im_v
                #plt.imshow(im_v)
                #return im_v
            elif (difx<0):
                difx = abs(difx)
                crop_img = img[(rows-difx):rows,0:cols]
                img = img[0:(rows-difx),0:cols]
                im_v = cv2.vconcat([crop_img, img])
                img = im_v
            else:
                img = img
                #plt.imshow(im_v)
            return img

    def box(self,img_box,rows,cols):
        h , w, _ = img_box.shape
        # row, col = 4, 3
        nh, nw = h//rows, w//cols
        # wb = 255 * np.ones(shape=[h,w, 3], dtype=np.uint8)
        for i in range(rows):    #for row 
            li = cv2.line(img_box,(0,nh*i),(w,nh*i),(0, 0, 0),5)
        for j in range(cols):
            li = cv2.line(img_box,(nw*j,0),(nw*j,h),(0, 0, 0),5)
        #plt.imshow(li)
        return li

def main():

    obj = Image()

    rows,cols = 5,5
    excel_path_1 = '/home/office5/Image_Code/Piyush_Sir/points_1.csv'
    df_1 = pd.read_csv(excel_path_1)

    excel_path_2 = '/home/office5/Image_Code/Piyush_Sir/points_3.csv'
    df_2 = pd.read_csv(excel_path_2)

    # Aligninig image
    
    path_1 = '/home/office5/Image_Code/Piyush_Sir/Images_p/'
    refx, refy = df_1['midx'].iloc[0], df_1['midy'].iloc[0]

    # For first CSV file
    crped_img = []
    for i in range(len(df_1.index)):
        img = df_1['File name'].iloc[i]
        img = f'{path_1}/{img}'
        x1, y1 = df_1['midx'].iloc[i], df_1['midy'].iloc[i]
        img = cv2.imread(img,0)
        crped_img.append(obj.align(refx,refy,x1,y1,img))

    hei,wid = crped_img[0].shape
    master_img = np.zeros([hei, wid])

    N = len(crped_img)

    for im in crped_img:
        imarr=np.array(im)
        master_img=master_img + imarr / N

    master_img_blk = []
    master_img_blk.append(obj.get_block(master_img,rows,cols))

    csv2_lst = []
    crped_img_2 = []
    for i in range(len(df_2.index)):
        img_2 = df_2['File name'].iloc[i]
        img_2 = f'{path_1}/{img_2}'
        img_2 = cv2.imread(img_2,0)
        x1, y1 = df_2['midx'].iloc[i], df_2['midy'].iloc[i]
        crped_img_2.append(obj.align(refx,refy,x1,y1,img_2))
        each_blk_2 = obj.get_block(crped_img_2[i],rows,cols)
        csv2_lst.append(each_blk_2)

    # Difference_lst_2 = []
    Difference_mst_blk = []
  
    for j in range(len(master_img_blk[0])):
        Difference_mst_blk.append(abs(obj.avg_master_diff(master_img_blk[0][j])))


    for i in range(len(csv2_lst)):
        m=0
        csv2_blk_lst = []
        for j in range(len(csv2_lst[0])):
            csv2_blk_lst.append(abs(obj.avg_block_diff(master_img_blk[0][j],csv2_lst[i][j])))
        # Difference_lst_2.append(csv2_blk_lst)
   
        img_name = df_2['File name'].iloc[i]
        img_2 = f'{path_1}/{img_name}'
        img_2 = cv2.imread(img_2)

        minMaxDiffThreashold = 120

        bg = obj.box(img_2,rows,cols)

        cen_hei = int(hei//rows) // 2
        cen_wid = int(wid//cols) // 2
        inc_hei = 0
        inc_wid = 0
        m=0
        for k in range(0,rows):
            for l in range(0,cols):

                thresold_block = 0.5
                if(abs(Difference_mst_blk[m])<minMaxDiffThreashold):
                    thresold_block = 0.1
                    
                if(csv2_blk_lst[m]<thresold_block):
                    sv = cv2.putText(bg, str(csv2_blk_lst[m])[0:6] + ":" + str(Difference_mst_blk[m]) , org=(abs(inc_wid + cen_wid), abs(inc_hei + cen_hei)),
                        fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=2, color=(0,0,0),
                        thickness=3, lineType=cv2.LINE_AA)
                else:
                    sv = cv2.putText(bg, (str(csv2_blk_lst[m])[0:6]+ ":" +str(Difference_mst_blk[m])) , org=(abs(inc_wid + cen_wid), abs(inc_hei + cen_hei)),
                        fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=2, color=(0,0,255),
                        thickness=3, lineType=cv2.LINE_AA)
                
                m = m+1

                inc_wid = inc_wid + int(wid//cols)

            inc_wid = 0
            inc_hei = inc_hei + int(hei//rows)

        cv2.imwrite(f'{path_1}/{img_name}_output1.png',sv)

if __name__=="__main__":
    main()